**Perform the Encoding**



In [ ]:
import numpy as np
from scipy import signal

# create a convolutional code with constraint length 3 and rate 1/2
g1 = np.array([5, 7], dtype=int)
g2 = np.array([7, 5], dtype=int)
ccode_3 = signal.convolve(g1, g2)

# create a convolutional code with constraint length 5 and rate 1/2
g1 = np.array([21, 27, 31, 23], dtype=int)
g2 = np.array([7, 25, 29, 19], dtype=int)
ccode_5 = signal.convolve(g1, g2)

# function to encode a message using a convolutional code
def conv_encode(message, ccode):
    encoded = []
    state = np.zeros(len(ccode)-1, dtype=int)
    for bit in message:
        in_bits = np.array([bit, state[0]], dtype=int)
        out_bits, state = signal.convolve(in_bits, ccode, 'full'), np.roll(state, 1)
        encoded.extend(list(out_bits))
    return np.array(encoded)
    
# generate random messages of length 100
message = np.random.randint(2, size=(100,))

# encode messages using the two different convolutional codes
encoded_3 = conv_encode(message, ccode_3)
encoded_5 = conv_encode(message, ccode_5)

print(f'Original message: {message}')
encoded = conv_encode(message, ccode_3)
print(f'Encoded message: {encoded}')


Original message: [1 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0
 0 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1
 1 1 1 0 0 1 1 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 1 1]
Encoded message: [35 74 35  0 35 74 35  0  0  0  0  0  0  0  0  0  0  0  0  0 35 74 35  0
 35 74 35  0 35 74 35  0 35 74 35  0 35 74 35  0  0  0  0  0 35 74 35  0
 35 74 35  0 35 74 35  0  0  0  0  0 35 74 35  0 35 74 35  0  0  0  0  0
  0  0  0  0 35 74 35  0 35 74 35  0 35 74 35  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0 35 74 35  0  0  0  0  0  0  0  0  0
 35 74 35  0  0  0  0  0  0  0  0  0 35 74 35  0 35 74 35  0  0  0  0  0
  0  0  0  0  0  0  0  0 35 74 35  0 35 74 35  0  0  0  0  0  0  0  0  0
 35 74 35  0 35 74 35  0  0  0  0  0 35 74 35  0 35 74 35  0 35 74 35  0
 35 74 35  0 35 74 35  0  0  0  0  0 35 74 35  0 35 74 35  0  0  0  0  0
 35 74 35  0 35 74 35  0  0  0  0  0 35 74 35  0 35 74 35  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0 

**Introduce Errors ad Decode with Viterbi.**


In [ ]:
# introduce increasing numbers of errors to the encoded strings
max_errors = 10
for num_errors in range(1, max_errors+1):
    # create a copy of the encoded strings to modify
    encoded_3_noisy = encoded_3.copy()
    encoded_5_noisy = encoded_5.copy()
    
    # randomly select num_errors indices to flip
    error_indices = np.random.choice(len(encoded_3), size=num_errors, replace=False)
    
    # flip the bits at the error indices
    for i in error_indices:
        encoded_3_noisy[i] = 1 - encoded_3_noisy[i]
        encoded_5_noisy[i] = 1 - encoded_5_noisy[i]
    
    # decode the noisy codewords using the Viterbi algorithm
    decoded_3 = signal.convolve(encoded_3_noisy, np.flip(ccode_3), mode='valid')
    decoded_5 = signal.convolve(encoded_5_noisy, np.flip(ccode_5), mode='valid')

     # compare the decoded messages to the original messages
    if np.array_equal(decoded_3, message) and np.array_equal(decoded_5, message):
        print(f'Successful decoding with {num_errors} errors.')
    else:
        print(f'Decoding failed with {num_errors} errors.')
        break


Decoding failed with 1 errors.


In [ ]:
 # compare the decoded messages to the original messages
print(f'Number of errors: {num_errors}')
print('Decoded messages (constraint length 3):', decoded_3)
print('Decoded messages (constraint length 5):', decoded_5)
print('')

Number of errors: 1
Decoded messages (constraint length 3): [7926 5180 2450 5180 7926 5180 1225    0    0    0    0    0    0    0
    0    0    0    0 1225 5180 7926 5180 2450 5180 7926 5180 2450 5180
 7926 5180 2450 5180 7926 5180 2450 5180 7926 5180 1225    0    0    0
 1225 5180 7926 5180 2450 5180 7926 5180 2450 5180 7926 5180 1225    0
    0    0 1225 5180 7926 5180 2450 5180 7926 5180 1225    0    0    0
    0    0    0    0 1225 5180 7926 5180 2450 5180 7926 5180 2450 5180
 7926 5180 1225    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0 1225 5180 7926 5180 1225    0
    0    0    0    0    0    0 1225 5180 7926 5180 1225    0    0    0
    0    0    0    0 1225 5180 7926 5180 2450 5180 7926 5180 1225    0
    0    0    0    0    0    0    0    0    0    0 1225 5180 7926 5180
 2450 5180 7926 5180 1225    0    0    0    0    0    0    0 1225 5180
 7926 5180 2450 5180 7926 5180 1225    0    0    0 1225 5180 7926 5180
 2450 5180 7926 5

In [ ]:
# function to simulate noisy channel and decode received message using Viterbi algorithm
def simulate_channel_and_decode(encoded, ccode):
    max_errors = 10
    for num_errors in range(1, max_errors+1):
        # create a copy of the encoded string to modify
        encoded_noisy = encoded.copy()

        # randomly select num_errors indices to flip
        error_indices = np.random.choice(len(encoded), size=num_errors, replace=False)

        # flip the bits at the error indices
        for i in error_indices:
            encoded_noisy[i] = 1 - encoded_noisy[i]

        # decode the noisy codeword using the Viterbi algorithm
        decoded = signal.convolve(encoded_noisy, np.flip(ccode), mode='valid')

        # compare the decoded message to the original message
        if np.array_equal(decoded, message):
            print(f'Successful decoding with {num_errors} errors.')
            return True
    print(f'Decoding failed with {max_errors} errors.')
    return False
# simulate noisy channel and decode received message using Viterbi algorithm for both codes
success_3 = simulate_channel_and_decode(encoded_3, ccode_3)
success_5 = simulate_channel_and_decode(encoded_5, ccode_5)

# compare failure rates of the two codes
if success_3 and not success_5:
    print("The failure rate of 'my' convolutional code is better than the control circuits.")
elif not success_3 and success_5:
    print("The failure rate of the control convolutional code is better than 'my' circuit.")
else:
    print("The failure rates of the two convolutional codes are the same.")

Decoding failed with 10 errors.
Decoding failed with 10 errors.
The failure rates of the two convolutional codes are the same.
